In [1]:
# In this script we analytically show that DEJMPS achieves the highest
# fidelity out of all bilocal Clifford protocols for all rho^{\otimes 2} with rho bell-diagonal.

import numpy as np
import itertools as it

Number of qubits

In [2]:
m = 2

# Load transversal
transversal_inv = load('2_transversal_inv.sobj')

Functions

In [3]:
def base(M, n):
    # calculate the image of the base under a matrix M
    
    # make a set of all combinations of the first column and the last n columns (these correspond to X_1, Z_1,...,Z_n)
    s = []
    for i in range(n+1, 2*n):
        s.append(M[0:2*n, i])
    powerset = it.chain.from_iterable(it.combinations(s, r) for r in range(1, len(s)+1))
    
    res = [vector(GF(2),2*n)]
        
    for i in powerset:
        v = vector(sum(i))     # calculate the sum of the elements of each combination (e.g IZZ = IZI + IIZ)
        res.append(v)
        
    return res

In [4]:
def pillars(M, n):
    # calculate the image of the pillars under a matrix M
    
    X1 = vector(M[0:2*n, 0])
    Z1 = vector(M[0:2*n, n])
    Y1 = X1 + Z1
    
    pI = base(M, n)
    pX = [(X1 + b) for b in pI]
    pY = [(Y1 + b) for b in pI]
    pZ = [(Z1 + b) for b in pI]
    
    return [pI, pX, pY, pZ]   

In [5]:
def tensor(A, n):
    # calculate the n fold tensor product of a matrix A
    
    kron = A
    count = 1
    while count < n:
        kron = np.kron(kron,A)
        count = count + 1
        
    if n == 2:
        res = np.reshape(kron, (4,4))
    elif n == 3:
        res = np.reshape(kron, (4,4,4)) 
    elif n == 4:
        res = np.reshape(kron, (4,4,4,4)) 
    elif n == 5:
        res = np.reshape(kron, (4,4,4,4,4)) 
        
    return res

In [6]:
def dist_stat(initial, M, n):
    # returns the success probability of an n-to-1 protocol M applied to an initial state
    pil = pillars(M, n)
    out = []
    for layer in pil:   
        coef = 0
        for elt in layer:
            if n == 2:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1])]
            if n == 3:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2])]
            if n == 4:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2]), int(elt[3]) + 2*int(elt[n+3])]
            if n == 5:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2]), int(elt[3]) + 2*int(elt[n+3]), \
                                    int(elt[4]) + 2*int(elt[n+4])]
        out.append(coef)   
    sp = sum(out)
    fid = out[0]/sp
    r = 1 # rate is set to one because we will be inputting variables and do not use it
    
    return sp, fid, r

In [7]:
# gets all possible tuples of success probabilities and fidelities.
# Prints the statistics corresponding to applying a CNOT_{12}, i.e. DEJMPS without rotations
# prints out the results twice since there is another protocol that achieves this,
# since there is the swap symmetry between the two rho states.

def sucprob_fid_lists(initial, transversal_inv, n):
    # calculate the possible distillation statistics (success probability & fidelity) of the protocols in a transversal
    # applied to an initial state
    
    for key, M in transversal_inv.items():
        break
        
    for i in range(4):
        for j in range(4):
            if i == j:
                M[i, j] = 1
            else:
                M[i, j] = 0
    i = 1
    j = 2
    
    M[j-1, i-1] = 1
    M[i-1+n, j-1+n] = 1
    
    s_DEJMPS = dist_stat(initial, M, 2)[0]
    f_DEJMPS = dist_stat(initial, M, 2)[1]
    tuple_DEJMPS = (s_DEJMPS, f_DEJMPS)
    
    fid = []
    sp = []
    fslist = []
    for key, M in transversal_inv.items():
        s = dist_stat(initial, M, n)[0]
        f = dist_stat(initial, M, n)[1]
        if (s,f) not in fslist:
            sp.append(s)
            fid.append(f)
            fslist.append((s,f))
        if (s, f) == tuple_DEJMPS:
            print('DEJMPS success prob and fidelity are:')
            print(s)
            print('--')
            print(f)
            print('--')
    return sp, fid

Results

In [8]:
# Bell diagonal state

a = var("a")
b = var("b")
c = var("c")

init = tensor(vector([1-a-b-c, a, b, c]), m)
all_Fs = sucprob_fid_lists(init, transversal_inv, m)[1]

DEJMPS success prob and fidelity are:
(a + b + c - 1)^2 + a^2 - 2*(a + b + c - 1)*b + b^2 + 2*a*c + c^2
--
((a + b + c - 1)^2 + b^2)/((a + b + c - 1)^2 + a^2 - 2*(a + b + c - 1)*b + b^2 + 2*a*c + c^2)
--
DEJMPS success prob and fidelity are:
(a + b + c - 1)^2 + a^2 - 2*(a + b + c - 1)*b + b^2 + 2*a*c + c^2
--
((a + b + c - 1)^2 + b^2)/((a + b + c - 1)^2 + a^2 - 2*(a + b + c - 1)*b + b^2 + 2*a*c + c^2)
--


In [12]:
DEJMPS = ((a + b + c - 1)^2 + b^2)/((a + b + c - 1)^2 + a^2 - 2*(a + b + c - 1)*b + b^2 + 2*a*c + c^2)
# copy the above rational function for the fidelity of DEJMPS

In [11]:
forget()
assume(0<a<1) # Assume coefficients are non-negative
assume(0<b<1) 
assume(0<c<1)

# assume coefficients are ordered a=>b=>c.
# This reordering can be done by local Pauli's, and is part of the original DEJMPS protocol
assume(a>b)
assume(b>c)
assume(a+b+c<1/2)

max_protocol = DEJMPS
for F in all_Fs:
    max_protocol = max_symbolic(max_protocol, DEJMPS).full_simplify()
#     max_protocol = max_symbolic(max_protocol, 0.9).full_simplify()
print("difference in fidelity between optimal and dejmps protocol:")
print((max_protocol-DEJMPS).full_simplify())

difference in fidelity between optimal and dejmps protocol:
0


In [ ]:
# Since the difference is zero, DEJMPS achieves the highest fidelity